In [2]:
%%writefile example2.txt
first 
second line
the third line
then a fourth line

Overwriting example2.txt


In [3]:
from pyspark import SparkContext

In [4]:
sc = SparkContext()

In [5]:
sc.textFile('example2.txt')

example2.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
textfile = sc.textFile('example2.txt')

In [7]:
words = textfile.map(lambda line: line.split())

In [8]:
words

PythonRDD[4] at RDD at PythonRDD.scala:53

In [9]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [10]:
textfile.collect()

['first ', 'second line', 'the third line', 'then a fourth line']

In [11]:
textfile.flatMap(lambda line: line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

In [12]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Overwriting services.txt


In [13]:
services = sc.textFile('services.txt')

In [14]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [15]:
splits = services.map(lambda line: line.split())

In [16]:
splits.take(2)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00']]

In [17]:
clean = services.map(lambda line: line[1:] if line[0] == '#' else line)

In [18]:
clean = clean.map(lambda split: split.split())

In [19]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [20]:
pairs = clean.map(lambda line: (line[3], line[-1]))

In [21]:
pairs.collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [22]:
pairs.reduceByKey(lambda x, y: float(x)+float(y)).collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [41]:
step1 = clean.map(lambda line: (line[3], line[-1]))
step2 = step1.reduceByKey(lambda x, y: float(x) + float(y))
step3 = step2.filter(lambda item: item[0] != 'State')
step4 = step3.sortBy(lambda item: item[1], ascending=False)

In [42]:
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]